In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [0]:
init_load_flag =dbutils.widgets.get("init_load_flag")

### Data _Reading from Source

In [0]:
df = spark.sql("select * from databricks_catalog.silver.customers_silver")

In [0]:
df.display()

In [0]:
df = df.dropDuplicates(subset=['customer_id'])
df.limit(10).display()

### **Dividing New Vs Old Records**

In [0]:
if init_load_flag == 0:
  df_old = spark.sql('''select DimCustomerkey,customer_id,create_date,update_date from databricks_catalog.gold.DimCustomer''')
else:
  df_old = spark.sql('''select 0 DimCustomerKey,0 customer_id,0 create_date,0 update_date from databricks_catalog.silver.customers_silver where 1=0''')

  df_old.display()


Applying join on df_old (on old records)

In [0]:
df_join = df.join(df_old,df.customer_id == df_old.customer_id,"left")

**Surrogate key-All the valaues**(we do not add surrogate key to all values)

In [0]:
df = df.withColumn("Dimcustomerkey", monotonically_increasing_id()+lit( 1))

In [0]:
df.limit(10).display()